# Complexidade de letras:

Este código foi desenvolvido para explorar e revelar a riqueza das letras musicais, proporcionando uma análise de sua complexidade lírica. Abaixo, apresentamos os critérios utilizados na análise:

Número de sílabas:
Palavras com múltiplas sílabas geralmente apresentam maior complexidade, tanto em termos de compreensão quanto de pronúncia.

Frequência:
A frequência das palavras na letra pode indicar sua acessibilidade e familiaridade para o público.

Análise gramatical:
Frases que utilizam muitos gerúndios tendem a ser mais coloquiais, enquanto aquelas que incorporam um maior número de conjunções subordinadas costumam ser mais formais.

Diversidade lexical:
A variedade de vocabulário e a riqueza semântica do texto contribuem para a profundidade e a expressividade das letras.

Importando todas as bibliotecas usadas:

In [3]:
import pandas as pd
import textstat as tst
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy as np

In [6]:
df = pd.read_csv("tcc_ceds_music.csv")

### Número de sílabas:

In [7]:
dic = {df.track_name[n]: {} for n in range(len(df))} #Para evitar que músicas de nomes iguais recebessem o mesmo número de sílabas, foi necessário colocar um index

for n in range(len(df)):
    lista = df.lyrics[n].split(" ")
    for palavra in lista:
        if palavra not in dic[df.track_name[n]]:
            dic[df.track_name[n]][palavra] = tst.syllable_count(palavra)

musicas_silabas_complexas = {}

for nome_musica, palavras in dic.items():
    for palavra, silabas in palavras.items():
        if silabas >= 6:
            if not any(palavra.count(letra) >= 4 for letra in set(palavra)): #Como em músicas é muito comum expressões como: "Ahhh", "Ohhh", foi necessário filtrá-las
                if nome_musica not in musicas_silabas_complexas:
                    musicas_silabas_complexas[nome_musica] = 1
                else:
                    musicas_silabas_complexas[nome_musica] += 1

In [8]:
df['silabas'] = df['track_name'].map(musicas_silabas_complexas).fillna(0)

Explicação: O código cria um dicionário (dic) onde as chaves são os nomes das músicas e os valores são dicionários que armazenam palavras e suas contagens de sílabas. Para cada música no DataFrame, as letras são divididas em palavras e, para cada palavra, o número de sílabas é contado utilizando a função tst.syllable_count. Em seguida, um segundo dicionário (musicas_silabas_complexas) é construído para identificar músicas com palavras que têm 6 ou mais sílabas, desde que essas palavras não contenham mais de três repetições de qualquer letra. Se uma palavra atende a esses critérios, o contador correspondente à música é incrementado. Assim, o código visa identificar e contar músicas que possuem palavras mais complexas em termos de sílabas.

### Frequência:

In [9]:
palavras_unicas_list = []
lista_ttr = []

for letra in range(len(df)):
    palavras = df.lyrics[letra].lower().split()
    palavras_unicas_list.append(len(set(palavras)))
    
for n in range(len(palavras_unicas_list)):
    lista_ttr.append(palavras_unicas_list[n]/df.len[n])

Explicação: O Type-Token Ratio (TTR) mede a diversidade lexical de um texto, representando a relação entre o número de palavras únicas e o total de palavra. Essa métrica permite comparar estilos de escrita entre diferentes letras de músicas ou autores e serve como indicador da complexidade linguística, onde letras mais desafiadoras tendem a ter um TTR maior.

### Análise Gramatical:

In [10]:
dic_tags = {track: 0 for track in df['track_name']}

for index, row in df.iterrows():
    tokens = word_tokenize(row['lyrics'].lower())
    tags = nltk.pos_tag(tokens)
    
    pontuacao = 0
    for palavra, tag in tags:

        if tag in ['NNP', 'NNPS']: #NNP: substantivo próprio, sigular. NNPS: substantivo próprio, plural
            pontuacao += 2
        elif tag in ['JJ', 'JJR', 'JJS']: #JJ: Adjetivo. JJR: adjetivo comparativo. JJS: adjetivo superlativo
            pontuacao += 1
        elif tag in ['RB', 'RBR', 'RBS']: #RB: advérbio. RBR: advérbio comparativo. RBS: advérbio superlativo.
            pontuacao += 1
        elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']: #VB: verbo. VBD: verbo passado. VBG: verbo gerúndio ou particípio presente. VBN: verbo particípio passado. VBP: verbo presente não 3ª pessoa singular. VBZ: verbo, presente, 3ª pessoa singular
            pontuacao += 1
        elif tag == 'WDT': #WDT: determinante interrogativo
            pontuacao += 1 
        elif tag == 'IN': # IN: preposição ou conjunção subordinativa 
            pontuacao += 1
            
    dic_tags[row['track_name']] = pontuacao

Explicação: Usando a biblioteca NLTK, é possível analisar palavra por palavra, determinando sua classe gramatical. Desse modo, as letras foram lidas e tiveram uma pontuação atribuída relacionada à presença de determinadas classes gramaticais que tornam o texto mais complexo

### Diversidade Lexical:

In [14]:
stop_words = set(stopwords.words('english'))

Para que o modelo consiga ler as letras, é necessário separá-las em listas, onde cada palavra estará separada por vírgulas. Além disso, para melhor eficácia do modelo, removi as stopwords.

In [15]:
letras = [[letra for letra in df['lyrics'][n].split() if letra not in stop_words] for n in range(len(df))]

Para o processamento, é necessário passar a lista como parâmetro, onde o modelo aprende as representações vetoriais das palavras com base em suas coocorrências no texto, resultando em um modelo que pode capturar relações semânticas e contextuais entre as palavras:

In [16]:
model = Word2Vec(letras, vector_size=100, window=5, min_count=1, workers=4)

Como o model leu todas as palavras presentes no dataset, ele consegue, baseado principalmente pela coocorência, transformar as palavras em vetores, facilitando a análise delas. Desse modo, para analisar a complexidade, irei usar a variabilidade:

In [17]:
def complexidade_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:
            vetores.append(modelo.wv[palavra]) #modelo.wv[palavra] retorna o vetor relacionado à palavra
            
    vetores_np = np.array(vetores)
    variancia = np.var(vetores_np, axis=0)

    return np.sum(variancia)

In [18]:
def calcular_complexidade(letra, modelo):
    letra_processada = [palavra.lower() for palavra in letra.split()]
    return complexidade_vetores(letra_processada, modelo)

In [20]:
dic_complexidade = {df.track_name[n]: {} for n in range(len(df))} 

for index, row in df.iterrows():
    letra = row['lyrics']
    complexidade = calcular_complexidade(letra, model)
    dic_complexidade[df.track_name[index]] = complexidade

In [ ]:
complexidades = [calcular_complexidade(letra, model, stop_words) for letra in df['lyrics']]
q1 = np.percentile(complexidades, 25)
q3 = np.percentile(complexidades, 75)
mediana = np.median(complexidades)

In [ ]:
for key, value in dic_complexidade.items():
    if value < q1:
        df.loc[df['track_name'] == key, "variabilidade"] = 1
    elif q1 < value < mediana:
        df.loc[df['track_name'] == key, "variabilidade"] = 2
    elif mediana < value < q3:
        df.loc[df['track_name'] == key, "variabilidade"] = 3
    else:
       df.loc[df['track_name'] == key, "variabilidade"] = 4